In [10]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

In [11]:
from unet_model import build_unet
model=build_unet()
model.to('cuda')

build_unet(
  (e1): encoder_block(
    (conv): conv_block(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (e2): encoder_block(
    (conv): conv_block(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (pool): MaxPool2d(kernel_size=(2, 2

In [12]:
loss_fn=nn.BCEWithLogitsLoss()
optimizer=optim.AdamW(model.parameters(),lr=0.0001)
scheduler=lr_scheduler.ReduceLROnPlateau(
    optimizer,mode='min',patience=8,factor=0.1,verbose=True
)

In [13]:
import pandas as pd
train_df=pd.read_csv('train_data.csv')
val_df=pd.read_csv('val_data.csv')

In [14]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
train_transforms = A.Compose([
        A.Resize(224,224, p=1.0),
        A.RandomBrightnessContrast(p=0.2),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        ToTensorV2(),
    ])

inference_transforms = A.Compose([
    A.Resize(224,224, p=1.0),
    ToTensorV2(),
])

In [15]:
from loader import MRISegmentationDataset
train_ds=MRISegmentationDataset(train_df,train_transforms)
val_ds=MRISegmentationDataset(val_df,inference_transforms)

In [16]:
from torch.utils.data import DataLoader
train_loader=DataLoader(
    train_ds,batch_size=4,num_workers=4,shuffle=True
)

In [17]:
val_loader=DataLoader(
    val_ds,batch_size=4,num_workers=4,shuffle=False  
)

In [18]:
from train import epochs
print(f'Train on {len(train_df)} samples, validate on {len(val_df)} samples.')
print('----------------------------------')

session_congif={
    'model'               : model,
    'train_loader'        : train_loader,
    'val_loader'          : val_loader,
    'optimizer'           : optimizer,
    'scheduler'           : scheduler,
    'loss_fn'             : loss_fn,
    'epochs'              : 100,
    'device'              : torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'),
    'save_path'           : 'Unet_Model.pth'
}

session_history=epochs(**session_congif)

Train on 2750 samples, validate on 412 samples.
----------------------------------


  0%|          | 0/100 [00:00<?, ?it/s]


Epoch 1/100


  1%|          | 1/100 [05:02<8:19:00, 302.43s/it]

loss: 0.2203 - dice_score: 0.3057 - eval_loss: 0.1153 - eval_dice_score: 0.4488 - lr: 0.0001

Epoch 2/100


...

loss: 0.0039 - dice_score: 0.8909 - eval_loss: 0.0056 - eval_dice_score: 0.8499 - lr: 1.0000000000000002e-06

Epoch 99/100


 99%|█████████▉| 99/100 [8:26:23<05:07, 307.82s/it]

loss: 0.0039 - dice_score: 0.8991 - eval_loss: 0.0054 - eval_dice_score: 0.8614 - lr: 1.0000000000000002e-06

Epoch 100/100


100%|██████████| 100/100 [8:31:31<00:00, 306.91s/it]

loss: 0.0038 - dice_score: 0.9038 - eval_loss: 0.0055 - eval_dice_score: 0.8548 - lr: 1.0000000000000002e-06
